In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, String, Float, DATE
import pymssql
from datetime import date, datetime
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
from empiricaldist import Cdf
import seaborn as sns
from pandasgui import show

ImportError: this platform is not supported: ('failed to acquire X connection: Bad display name ""', DisplayNameError(''))

Try one of the following resolutions:

 * Please make sure that you have an X server running, and that the DISPLAY environment variable is set correctly

In [3]:
def find_host_ip():
    cmd = "ipconfig.exe | grep 'vEthernet (WSL)' -A4 | cut -d':' -f 2 | tail -n1 | sed -e 's/\s*//g' > host_ip.txt"
    os.system(cmd)
    with open('host_ip.txt') as f:
        HOST_IP = f.readline()
        f.close()
    return HOST_IP.split(sep='\n')[0]

In [4]:
HOST_IP = find_host_ip()

In [5]:
pd.set_option('display.max_columns', 10)

env_path = os.path.join(r'/home/emad/code/emadam/glassdoor/glassdoor/', 'db_login.env')
env_path

'/home/emad/code/emadam/glassdoor/glassdoor/db_login.env'

In [6]:
if os.path.exists(env_path):
    load_dotenv(env_path)
SERVER_NAME = os.getenv('server')
DB_NAME = os.getenv('database')
USERNAME = os.getenv("pymssql_username")
PASSWORD = os.getenv("pymssql_password")

In [7]:
SERVER_NAME, DB_NAME, USERNAME, PASSWORD, HOST_IP

('DESKTOP-TNNDC3L', 'Glassdoor', 'sa', 'Emad1983', '172.21.192.1')

In [8]:
class scraper(): 
    
    def __init__(self, soup):
        self.soup = soup
        pass
    
    def extract_job_title_from_result(soup):
        jobs = []
        for a in soup.find_all(name="a", attrs={"data-test": "job-link"}):
            for spans in a.find_all(name="span"):
                jobs.append(spans.text)
        return jobs


    def extract_company_name_from_result(soup):
        coname = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka0"}):
            for div in div.find_all(name="div", attrs={"class": "d-flex justify-content-between align-items-start"}):
                for a in div.find_all(name="a"):
                    for spans in a.find_all(name="span"):
                        coname.append(spans.text)
        return coname


    def extract_company_rate_from_result(soup):
        corate = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka1"}):
            if div.find_all(name="span", attrs={"class": "e1cjmv6j0"}):
                for spans in div.find_all(name="span", attrs={"class": "e1cjmv6j0"}):
                    corate.append(spans.text)
            else:
                corate.append(np.nan)
        return corate


    def extract_company_location_from_result(soup):
        coloc = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka2"}):
            for spans in div.find_all(name="span"):
                coloc.append(spans.text)
        return coloc


    def extract_company_salary_from_result(soup):
        cosal = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka0"}):
            if div.find_all(name="div", attrs={"class": "e1rrn5ka3"}):
                for spans in div.find_all(name="span", attrs={"data-test": "detailSalary"}):
                    cosal.append(spans.text)
            else:
                cosal.append(np.nan)
        return cosal


    def extract_job_age_from_result(soup):
        jobage = []
        for div in soup.find_all(name='div', attrs={"class": ["e1rrn5ka2", "e1rrn5ka3"]}):
            for age in div.find_all(name='div', attrs={"data-test": "job-age"}):
                result = age.text
                result = result.replace('24h', '1d')
                result = result.replace('d', '')
                result = result.replace('30+', '31')
                t_dif = np.timedelta64(result, 'D')
                if t_dif < np.timedelta64(31, 'D'):
                    ad_date = np.datetime64(date.today()) - t_dif
                    ad_date = ad_date.astype(datetime)
                    jobage.append(ad_date)
                else:
                    jobage.append(np.nan)
        return jobage


    def job_seniority(job):
        if job.find('Senior') != -1:
            return 'Senior'
        if job.find('Junior') != -1:
            return 'Junior'
        if job.find('Entry level') != -1:
            return 'Entry level'
        if job.find('Graduate') != -1:
            return 'Graduate'
        if job.find('Manager') != -1:
            return 'Manager'
        if job.find('Internship') != -1:
            return 'Internship'
        else:
            return np.nan


    def extract_job_link_from_result(soup):
        joblink = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka0"}):
            for a in div.find_all(name='a', href=True):
                joblink.append('glassdoor.com.au'+a['href'])
            else:
                joblink.append(np.nan)
        return joblink

In [9]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/92.0.4515.159 Safari/537.36"}
URL = f'https://www.glassdoor.com.au/Job/melbourne-junior-data-analyst-jobs-SRCH_IL.0,9_IC2264754_KO10,29.htm'
resp = requests.get(URL, headers=headers)
# specifying a desired format of page using the html parser
soup = BeautifulSoup(resp.text, "html.parser")

In [10]:
job_title = scraper.extract_job_title_from_result(soup)
co_name = scraper.extract_company_name_from_result(soup)
co_rate = scraper.extract_company_rate_from_result(soup)
co_loc = scraper.extract_company_location_from_result(soup)
co_sal = scraper.extract_company_salary_from_result(soup)
job_age = scraper.extract_job_age_from_result(soup)
# job_link = scraper.extract_job_link_from_result(soup)

In [11]:
data = list(zip(job_title, co_name, co_rate, co_loc, co_sal, job_age))#, job_link))
job_data = pd.DataFrame(data)
job_data = job_data.rename(columns={0: 'Job Title', 1: 'Company', 2: 'Rank', 3: 'Location', 4: 'Salary', 5: 'Ad Date'}) #,6: 'Hyperlink'})
job_data['Ad Date'] = pd.to_datetime(job_data['Ad Date'])

In [12]:
job_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Job Title  30 non-null     object        
 1   Company    30 non-null     object        
 2   Rank       26 non-null     object        
 3   Location   30 non-null     object        
 4   Salary     19 non-null     object        
 5   Ad Date    24 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 1.5+ KB


In [13]:
try:
    conn = pymssql.connect(host=HOST_IP, port=1433 , server=SERVER_NAME, database=DB_NAME, user=USERNAME, password=PASSWORD)
    #     conn = pymssql.connect(host='localhost', port=1433 , server=SERVER_NAME, database=DB_NAME, user=USERNAME, password=PASSWORD)
except pymssql.Error as err:
    print(("[ERROR] {}".format(err))) #172.21.192.1

In [14]:
cursor = conn.cursor()
cursor.execute('SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES;')
row = cursor.fetchall()
row

[('jobs_final',), ('jobs',), ('jobs_temp',)]

In [15]:
SQL_Query = pd.read_sql_query(
    '''select *
        from [Glassdoor].[dbo].[jobs]''', conn)
pd.DataFrame(SQL_Query)

/home/emad/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Job Title,Company,Rank,Location,Salary,Ad Date
0,Start your new career as a Data Analyst,Data School Down Under,4.5,Melbourne,"A$50,000 - A$60,000 (Employer Est.)",None
1,Entry Level Data & Reporting Analyst,Global 360,3.5,Doncaster,None,2021-09-09
2,Fraud Data & Reporting Analyst,AGL Energy,3.4,Melbourne,None,2021-09-09
3,Investment Data Analyst,Vanguard,3.6,Melbourne,None,2021-09-09
4,Data Analyst,Each,3.0,Ringwood,"A$190,800 (Employer Est.)",2021-09-09
...,...,...,...,...,...,...
1046,Junior Financial Analyst (Graduate),BMYG Corporate Pty Ltd,NaN,Box Hill,"A$55,000 (Employer Est.)",2022-03-01
1047,Junior Test Analyst,FinXL,3.4,Melbourne,None,2022-03-19
1048,Master Data Analyst,Talent International,4.6,Melbourne,None,2022-03-16
1049,People Success Officer,Australian Spatial Analytics,NaN,Melbourne,"A$70,000 (Employer Est.)",2022-03-18


In [16]:
USERNAME, PASSWORD, DB_NAME, HOST_IP

('sa', 'Emad1983', 'Glassdoor', '172.21.192.1')

In [17]:
engine = create_engine(f'mssql+pymssql://{USERNAME}:{PASSWORD}@{HOST_IP}:1433/{DB_NAME}', pool_pre_ping=True)

In [18]:
job_data.to_sql("jobs_temp", engine, if_exists='replace', index=False)
query = """
        SELECT * FROM jobs_temp
        EXCEPT
        SELECT * FROM jobs;
    """
new_entries = pd.read_sql(query, engine)

In [19]:
if ('jobs',) not in list(row):
    job_data.to_sql("jobs", engine, if_exists='replace', index=False, dtype={
        'Job Title': String(255),
        'Company': String(255),
        'Rank': Float(),
        'Location': String(255),
        'Salary': String(255),
        'Ad Date': DATE()#,
        #'Hyperlink': String(255)
    })
else:
    job_data.to_sql("jobs_temp", engine, if_exists='replace', index=False)
    query = """
        SELECT * FROM jobs_temp
        EXCEPT
        SELECT * FROM jobs;
    """
    new_entries = pd.read_sql(query, engine)

    new_entries.to_sql("jobs", engine, if_exists='append', index=False, dtype={
        'Job Title': String(255),
        'Company': String(255),
        'Rank': Float(),
        'Location': String(255),
        'Salary': String(255),
        'Ad Date': DATE()#,
        #'Hyperlink': String(255)
    })



In [20]:
SQL_Query = pd.read_sql_query(
    '''select *
        from [Glassdoor].[dbo].[jobs]''', conn)

/home/emad/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [21]:
jobs_stored = pd.DataFrame(SQL_Query)
jobs_stored.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Job Title  1059 non-null   object 
 1   Company    1059 non-null   object 
 2   Rank       955 non-null    float64
 3   Location   1059 non-null   object 
 4   Salary     678 non-null    object 
 5   Ad Date    937 non-null    object 
dtypes: float64(1), object(5)
memory usage: 49.8+ KB


In [22]:
jobs_stored['Ad Date'] = pd.to_datetime(jobs_stored['Ad Date'])
jobs_stored.sort_values(by=['Ad Date'], inplace=True)
jobs_stored.drop_duplicates(subset=['Job Title', 'Company', 'Location'], keep='first', inplace=True)
ad_count = jobs_stored.groupby('Ad Date').size()
jobs_stored = jobs_stored.set_index(pd.DatetimeIndex(jobs_stored['Ad Date'])).sort_index()


In [23]:
jobs_stored.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 2021-08-13 to NaT
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Job Title  265 non-null    object        
 1   Company    265 non-null    object        
 2   Rank       233 non-null    float64       
 3   Location   265 non-null    object        
 4   Salary     166 non-null    object        
 5   Ad Date    245 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 14.5+ KB


In [24]:
import regex as re

In [25]:
def salary_convert(salary):
    if salary == 0:
        return np.nan
    if salary < 1000:
        return salary * 1788
    else:
        return salary

In [26]:
jobs_stored['Min_Salary'] = jobs_stored['Salary'].str.extract(r'([0-9]+,*[0-9]+).*')
jobs_stored['Min_Salary'] = jobs_stored['Min_Salary'].str.replace(r'\,', '', regex=True).astype(float).astype(
    pd.Int64Dtype())

jobs_stored['Max_Salary'] = jobs_stored['Salary'].str.extract(r'[0-9]+,*[0-9]+.*?([0-9]+,*[0-9]+)')
jobs_stored['Max_Salary'] = jobs_stored['Max_Salary'].str.replace(r'\,', '', regex=True).astype(float).astype(
    pd.Int64Dtype())

jobs_stored['Min_Salary'] = jobs_stored['Min_Salary'].fillna(value=0)
jobs_stored_min = jobs_stored.apply(lambda x: salary_convert(x['Min_Salary']), axis=1)
jobs_stored['Min_Salary'] = pd.DataFrame(jobs_stored_min)

jobs_stored['Max_Salary'] = jobs_stored['Max_Salary'].fillna(value=0)
jobs_stored_max = jobs_stored.apply(lambda x: salary_convert(x['Max_Salary']), axis=1)
jobs_stored['Max_Salary'] = pd.DataFrame(jobs_stored_max)

jobs_stored[['Salary', 'Min_Salary', 'Max_Salary']]

,Salary,Min_Salary,Max_Salary
Ad Date,,,
2021-08-13,None,NaN,NaN
2021-08-13,None,NaN,NaN
2021-08-19,None,NaN,NaN
2021-08-20,None,NaN,NaN
2021-08-20,None,NaN,NaN
...,...,...,...
NaT,None,NaN,NaN
NaT,"A$60,000 - A$105,000 (Glassdoor Est.)",60000.0,105000.0
NaT,A$63 - A$88 Per Hour(Employer Est.),112644.0,157344.0


In [27]:
jobs_stored['Seniority'] = jobs_stored['Job Title'].apply(lambda x: 'Senior' if x.find('Senior') != -1 else
('Junior' if x.find('Junior') != -1 else ('Entry Level' if x.find('Entry level') != -1 else
                                          ('Graduate' if x.find('Graduate') != -1 else
                                           ('Manager' if x.find('Manager') != -1 else
                                            ('Internship' if x.find('Internship') != -1 else np.nan))))))
jobs_stored.dropna(subset=['Ad Date'], how='all', inplace=True)
jobs_stored[['Job Title', 'Seniority']]

,Job Title,Seniority
Ad Date,,
2021-08-13,Data Analyst,NaN
2021-08-13,Data Analyst,NaN
2021-08-19,Data Analyst,NaN
2021-08-20,Data Analyst,NaN
2021-08-20,Data Analyst,NaN
...,...,...
2022-03-24,.Net Developer,NaN
2022-03-26,Workplace Experience Research - Consultant (Me...,NaN
2022-03-27,Junior Merchandise Planner - Toys,Junior


In [28]:
min_sal = np.array(jobs_stored['Min_Salary'].dropna())
sal_replicates = np.empty(10000)
for i in range(10000):
    sal_samples = np.random.choice(min_sal, len(min_sal))
    sal_replicates[i] = np.mean(sal_samples)
con_interval = np.percentile(sal_replicates, [2.5, 97.5])
con_interval[0] = ('{:.1f}'.format(con_interval[0]))
con_interval[1] = ('{:.1f}'.format(con_interval[1]))
print("Minimum Salary 95% Confidence Interval = " + str(con_interval))

Minimum Salary 95% Confidence Interval = [80586.6 87814.8]


In [29]:
jobs_stored.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 245 entries, 2021-08-13 to 2022-03-30
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Job Title   245 non-null    object        
 1   Company     245 non-null    object        
 2   Rank        214 non-null    float64       
 3   Location    245 non-null    object        
 4   Salary      151 non-null    object        
 5   Ad Date     245 non-null    datetime64[ns]
 6   Min_Salary  151 non-null    float64       
 7   Max_Salary  141 non-null    float64       
 8   Seniority   49 non-null     object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 19.1+ KB


In [35]:
show(jobs_stored)

NameError: name 'show' is not defined

In [ ]:
plt.style.use('seaborn-whitegrid')
sns.set()
fig, ax = plt.subplots(2, 2)
fig.set_size_inches(16, 11)
min_salary = jobs_stored['Min_Salary']
before_Date = jobs_stored['Ad Date'] < pd.to_datetime('2021-10-15')
ax[0, 0].plot(Cdf.from_seq(min_salary[before_Date].dropna()), label='Before 2021 October 15')
ax[0, 0].plot(Cdf.from_seq(min_salary[~before_Date].dropna()), label='After 2021 October 15')
x_min = np.sort(jobs_stored['Min_Salary'].dropna())
y_min = np.arange(1, len(x_min) + 1) / len(x_min)
x_max = np.sort(jobs_stored['Max_Salary'].dropna())
y_max = np.arange(1, len(x_max) + 1) / len(x_max)
pct_list = np.array([25, 50, 75])
maxpct_val = np.percentile(jobs_stored['Max_Salary'].dropna(), pct_list)
minpct_val = np.percentile(jobs_stored['Min_Salary'].dropna(), pct_list)
ax[0, 0].set_ylabel('CDF')
ax[0, 0].set_title('Distribution of minimum salary of "Data Analyst" jobs on Glassdoor')
ax[0, 0].legend()
ax[0, 0].set_xlabel('Estimated salary')

ax[0, 1].plot(x_min, y_min, marker='.', linestyle='none', color='r', label='Minimum salary')
ax[0, 1].plot(x_max, y_max, marker='.', linestyle='none', color='b', label='Maximum salary')
ax[0, 1].plot(maxpct_val, pct_list / 100, marker='^', linestyle='none', color='c',
              label='25th, 50th and 75th Percentile')
ax[0, 1].plot(minpct_val, pct_list / 100, marker='^', linestyle='none', color='k',
              label='25th, 50th and 75th Percentile')
ax[0, 1].annotate('Mean:', xy=(jobs_stored['Min_Salary'].mean().astype('int64'), 0.5), xytext=(40000, 0.9),
                  arrowprops=dict(arrowstyle="fancy", facecolor='green', connectionstyle="angle3,angleA=0,angleB=-90"), )
ax[0, 1].set_ylabel('ECDF')
ax[0, 1].set_title('Distribution of min and max salary of "Data Analyst" on Glassdoor')
ax[0, 1].legend()
ax[0, 1].set_xlabel('Estimated salary')

ax[1, 0].bar(jobs_stored.index.unique(), ad_count, linestyle='None', color='r')
ax[1, 0].figure.canvas.draw()
ax[1, 0].tick_params(axis='x', which='major', rotation=20, direction='inout', length=6, width=2,
                     color='k', labelcolor='royalblue')
ax[1, 0].set_xlabel('Date of Advertisement', labelpad=0.0, color='magenta')
ax[1, 0].set_ylabel('Number of Ads', color='purple')
ax[1, 0].set_title('\'Data Analyst Job\' Advertisements in Glassdoor website', color='limegreen')

ax[1, 1].pie(jobs_stored['Seniority'].value_counts(), labels=jobs_stored['Seniority'].dropna().unique(),
             normalize=True, autopct='%1.1f%%', shadow=True, startangle=0)
ax[1, 1].set_title('Job Ads seniority level count')
# plt.tight_layout()
fig.savefig("glassdoor" + np.datetime64(date.today()).astype('str') + ".png")